In [1]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import undetected_chromedriver as uc
from selenium_stealth import stealth
from selenium.webdriver.common.by import By

In [2]:
HOME_ADDRESS = "https://historicalnewspapers.lib.purdue.edu/?a=cl&cl=CL1&sp=PE&e=-------en-20--1--txt-txIN--------1"
options = Options()
#options.add_argument("--headless")
driver = uc.Chrome(options=options)
stealth(driver,
                languages=["en-US", "en"],
                vendor="Google Inc.",
                platform="Win32",
                webgl_vendor="Intel Inc.",
                renderer="Intel Iris OpenGL Engine",
                fix_hairline=True
        )

driver.implicitly_wait(5)

In [3]:
driver.get(HOME_ADDRESS)
input("Enter anything when captcha complete")
print(driver.title)

Enter anything when captcha complete 


The Purdue Exponent (1889-2017) — Browse by title — Purdue University Newspapers Collection


In [4]:
start_year = 1939

In [5]:
calendar = driver.find_element(By.ID, 'datebrowserrichardtoplevelcalendar')

In [6]:
year = driver.find_element(By.XPATH, f"//h2[contains(text(), '{start_year}')]/following-sibling::div[1]")

In [7]:
first_link = year.find_element(
    By.XPATH,
    "//ul[contains(@class,'nav') and contains(@class,'nav-pills')]//a[@class='nav-link'][1]"
)
first_link.click()

In [10]:
calendar_box = driver.find_element(By.ID, "datebrowserrichardmonthlevelcalendar")
weeks = calendar_box.find_elements(By.XPATH, "./*[position() > 1]")

links = []

In [ ]:
for week in weeks:
    days = week.find_elements(By.XPATH, "child::*")
    for day in days:
        try:
            date = day.find_element(By.CSS_SELECTOR, "b.hiddenwhennotsmall").get_attribute("innerHTML")
        except: break
        try:
            brow = day.find_element(By.CSS_SELECTOR, "div.card").text
            print(date)
            print(brow)
        except: continue
            #day.find_element(By.CSS_SELECTOR, "div.card a")
            # if card_links:
            #     links.append(card_links[0].get_attribute("href"))

Sunday, 15 December 1889
The Purdue Exponent


In [14]:
print(links)

[]
